# 실습_셀프 어텐션을 이용한 텍스트 분류

트랜스포머는 RNN 계열의 seq2seq를 대체하기 위해 등장했다. 트랜스포머의 인코더는 RNN인코더를, 트랜스포머의 디코더는 RNN 디코더를 대체할 수 있다. 

트랜스포머의 인코더는 셀프 어텐션이라는 매커니즘을 통해 문장을 이해한다. 트랜스포머의 인코더는 다양한 분야의 자연어 처리 태스크에서 사용될 수 있었고, 이 아이디어는 BERT라는 모델로 이어지게 된다. 여기서는 트랜스포머의 인코더를 사용하여 텍스트 분류를 수행한다. 

## 1. 멀티 헤드 어텐션

In [19]:
import tensorflow as tf

In [20]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

## 2. 인코더 설계하기

In [21]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

## 3. 포지션 임베딩

In [22]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

## 4. 데이터 로드 및 전처리

In [30]:
vocab_size = 20000   # 상위 20000개 단어만 고려하겠다. 
maxlen = 200

(X_train,y_train),(X_test,y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

In [31]:
print(len(X_train[0]))
print(len(X_test[0]))

218
68


In [33]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [34]:
print(len(X_train[0]))
print(len(X_test[0]))

200
200


In [42]:
X_train

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

## 5. 트랜스포머를 이용한 IMDB 리뷰 분류

In [36]:
embedding_dim = 32  # 각 토큰을 표현할 벡터의 차원 수
num_heads = 2       # 어텐션 헤드의 개수
dff = 32            # 트랜스포머 내부의 피드 포워드 네트워크의 hidden layer수

inputs = tf.keras.layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [38]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
token_and_position_embedding (None, 200, 32)           646400    
_________________________________________________________________
transformer_block (Transform (None, 200, 32)           6464      
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0     

In [40]:
model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train,y_train,batch_size=32, epochs=2, validation_data=(X_test,y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.3835 - accuracy: 0.8177 - val_loss: 0.2968 - val_accuracy: 0.8746
Epoch 2/2
25000/25000 [==============================] - 81s 3ms/sample - loss: 0.2014 - accuracy: 0.9227 - val_loss: 0.3133 - val_accuracy: 0.8665


In [41]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

25000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

테스트 정확도: 0.8665


<br><br>
# 클래스 사용하기

클래스는 객체를 표현하기 위한 문법이다. 게임에서 사용하는 마법사, 궁수나 스크롤 바, 버튼, 체크 박스처럼 특정한 개념이나 모양으로 존재하는 것을 객체라고 부른다. 게임의 기사 캐릭터를 표현하기 위해 사용하는 체력, 공격력, 주문력 등을 클래스의 속성attribute라고 부르고, 베기, 찌르기 등의 기능을 메서드라고 부른다. 파이썬에서 숫자,문자,리스트,딕셔너리 모든 것이 객체이다. 

## 1. 클래스와 메서드 만들기

In [43]:
# 메서드의 첫 번째 매개변수는 반드시 self를 지정해야 한다. 
class Person:
    def greeting(self):
        print('Hello')

In [44]:
# 인스턴스 = 클래스()
# james는 Person의 인스턴스이다. 
james = Person()

james.greeting()

Hello


In [47]:
# 메서드 안에서 메서드를 호출할 때는 self.메서드()형식으로 호출해야한다. 
class Person():
    def greeting(self):
        print('Hello')
    
    def hello(self):
        self.greeting()
        
james = Person()
james.hello()

Hello


## 2. 속성 사용하기

In [49]:
# 속성을 만들때는 __init__ 메서드 안에서 self.속성에 값을 할당한다. 
# __init__ 메서드는 인스턴스를 만들때 호출되는 특별한 메서드이다. 
# initailize 라는 이름 그대로 인스턴스를 초기화 한다.

class Person:
    def __init__(self):
        self.hello = '안녕하세요'
    
    def greeting(self):
        print(self.hello)
        
james = Person()
james.greeting()

안녕하세요


## 3. 인스턴스 만들 때 값 받기

In [51]:
# self.name처럼 매개변수를 그대로 self에 넣어서 속성으로 만들었다. 
class Person:
    def __init__(self, name, age, address):
        self.hello = '안녕하세요'
        self.name = name
        self.age = age
        self.address = address
        
    def greeting(self):
        print('{0} 저는 {1}입니다'.format(self.hello, self.name))
        
maria = Person('마리아', 20, '서울')
maria.greeting()

print(maria.name)
print(maria.age)
print(maria.address)

안녕하세요 저는 마리아입니다
마리아
20
서울
